In [1]:
from src.Slm import slm
import numpy as np
import matplotlib.pyplot as plt
import csv

# Loading parmaters
E = 200e9 # Elastic modulus (Pa)
D = 1*10e-3 # Diameter mm
P = 7850 # Density (kg/m^3)
F = [0,-1,0,0,0,0,0,0]

# Mechanism Parameters
a0 = .1
Rw0 = a0/2
Rh0 = (a0/2)*np.sqrt(3)
Rr0 = 1
Rf0 = False

# Simulation parameters
step_size = a0/10
Rr_count = 30

# Path to csv results
file_name = "./results/simulation/sim_results4.csv"

# Create new file with column headers
with open(file_name, 'w', newline='') as csvfile:
    slm_writer = csv.writer(csvfile, delimiter=' ',
            quotechar='|', quoting=csv.QUOTE_MINIMAL)
    slm_writer.writerow(["Rr","Rf","Rh","Rw","A","k_max","k_min","Range","Mass"])

# Define ranges to sweep through in simulation
Rh_range = np.arange(0.1*a0,3*a0,step_size)
Rw_range = np.arange(-5*a0,3*a0,step_size)
Rr_range = np.linspace(0.2,1,Rr_count)

# Normalizes system properties based on linkage values
def normailize_properties(k_max,k_min,range,mass):
    return (k_max*a0)/(D**2),(k_min*a0)/(D**2),(range/a0),(mass/(D**2*a0))

# Collects and writes properties of mechanisms
def write_properties(SLM,Rr,Rf,A,Rh,Rw):
    try:
        SLM.update_links(Rr,Rf,A=A,Rh=Rh,Rw=Rw)
        k_max,k_min,range,mass = SLM.find_system_properties(F,E,D,P)
        k_max,k_min,range,mass = normailize_properties(k_max,k_min,range,mass)
        with open(file_name, 'a', newline='') as csvfile:
            slm_writer = csv.writer(csvfile, delimiter=' ',
                    quotechar='|', quoting=csv.QUOTE_MINIMAL)
            slm_writer.writerow([Rr,Rf,Rh,Rw,A,k_max,k_min,range,mass])
        return k_max,k_min,range,mass
    except Exception as e: print("Error encountered:",e,"Params:",Rr,Rf,A,Rh,Rw)

print(len(Rh_range),len(Rw_range),len(Rr_range))

# Simulate mechanism across full design space
def full_sweep():
    # Define base mechanism
    SLM = slm.mechanism(Rr=Rr0,Rf=False,A=a0,Rh=Rh0,Rw=Rw0)
    # Full Mechanism Sweep
    for Rh in Rh_range:
        for Rw in Rw_range:
            for Rr in Rr_range:
                if not (Rw<0.1*a0 and Rw>-0.1*a0) and not (Rw<-.9*a0 and Rw>-1.1*a0):
                    Rf = False
                    write_properties(SLM,Rr,Rf,a0,Rh,Rw)
                    if Rr != 1:
                        Rf = True
                        write_properties(SLM,Rr,Rf,a0,Rh,Rw)


# Run simulation
full_sweep()

30 80 30
